In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import math
import json
import pickle
import tensorflow.python.keras as keras
from keras.layers import Input, Layer, LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Add, Lambda
from keras.models import Model, load_model, model_from_json, clone_model
from keras.optimizers import SGD, Adam
import keras.backend as K
from PIL import Image
import os
K.clear_session()
'''
# file h5 dibuat dengan convert.py dari https://github.com/awe777/keras-yolo3
model0 = load_model("yolov3_tiny.h5")   # dibuat dari kode sebelum fork
model1 = load_model("yolov3_tiny_.h5")  # dibuat dari kode setelah diubah di fork pribadi
print(model0.to_json() == model1.to_json())
print(model.summary())
# '''
print()

Using TensorFlow backend.


In [2]:
def roundingAlgo(x):
    return K.round(x)

class RoundQinf_4(Layer):
    def __init__(self, **kwargs):
        super(RoundQinf_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundQinf_4, self).build(input_shape)
    def call(self, X):
        return roundingAlgo(X * 16) / 16.0
    def get_config(self):
        base_config = super(RoundQinf_4, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ7_12(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ7_12, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ7_12, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 4096) + 524288) % 1048576) - 524288) / 4096.0
    def get_config(self):
        base_config = super(RoundOverflowQ7_12, self).get_config()
        return dict(list(base_config.items()))
class RoundQx_8minusx(Layer):
    def __init__(self, **kwargs):
        super(RoundQx_8minusx, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundQx_8minusx, self).build(input_shape)
    def call(self, X):
        pow_2_to_expplus8 = 2 ** (8 + (K.log(K.abs(X)) / math.log(2)) // 1)
        return roundingAlgo(X * pow_2_to_expplus8) / (pow_2_to_expplus8 * 1.0)
    def get_config(self):
        base_config = super(RoundQx_8minusx, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ3_4(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ3_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ3_4, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 16) + 128) % 256) - 128) / 16.0
    def get_config(self):
        base_config = super(RoundOverflowQ3_4, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ7_0(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ7_0, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ7_0, self).build(input_shape)
    def call(self, X):
        return ((roundingAlgo(X) + 128) % 256) - 128
    def get_config(self):
        base_config = super(RoundOverflowQ7_0, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ7_4(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ7_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ7_4, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 16) + 2048) % 4096) - 2048) / 16.0
    def get_config(self):
        base_config = super(RoundOverflowQ7_4, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ11_4(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ11_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ11_4, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 16) + 32768) % 65536) - 32768) / 16.0
    def get_config(self):
        base_config = super(RoundOverflowQ11_4, self).get_config()
        return dict(list(base_config.items()))
class Identity(Layer):
    def __init__(self, **kwargs):
        super(Identity, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        base_config = super(Identity, self).get_config()
        return dict(list(base_config.items()))
class IdentityFinalLayer(Layer):
    def __init__(self, **kwargs):
        super(IdentityFinalLayer, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        base_config = super(IdentityFinalLayer, self).get_config()
        return dict(list(base_config.items()))

print("Custom layer classes successfully defined")

Custom layer classes successfully defined


In [3]:
customObjects = {
    'Identity': Identity,
    'RoundQinf_4': RoundQinf_4,
    'RoundOverflowQ3_4': RoundOverflowQ3_4,
    'RoundQx_8minusx': RoundQx_8minusx,
    'RoundOverflowQ7_12': RoundOverflowQ7_12,
    'RoundOverflowQ7_0': RoundOverflowQ7_0,
    'RoundOverflowQ7_4': RoundOverflowQ7_4,
    'RoundOverflowQ11_4': RoundOverflowQ11_4,
    'IdentityFinalLayer': IdentityFinalLayer
}

In [4]:
def prepare_image(image_dir, input_shape):
    # from PIL import Image
    # import numpy as np
    image_data = 0
    h, w = input_shape
    try:
        image = Image.open(image_dir)
        iw, ih = image.size
        scale = min(w/iw, h/ih)
        nw = int(iw*scale)
        nh = int(ih*scale)
        dx = (w-nw)//2
        dy = (h-nh)//2
        image = image.resize((nw,nh), Image.BICUBIC)
        new_image = Image.new('RGB', (w,h), (128,128,128))
        new_image.paste(image, (dx, dy))
        image_data = np.array(new_image)/255.
        return image_data, image.size[::-1]
    except:
        return None, None
# TO DO: implement yolo_head in order to get human-understandable result from model output

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

tinyYolo_anchors = get_anchors("../CNN-VLSI/tiny_yolo_anchors.txt")

def yolo_head_nonLoss(feats, anchors, num_classes, input_shape):
    """Convert final layer features to bounding box parameters."""
    num_anchors = len(anchors)
    # Reshape to batch, height, width, num_anchors, box_params.
    anchors_tensor = K.reshape(K.constant(anchors), [1, 1, 1, num_anchors, 2])

    grid_shape = K.shape(feats)[1:3] # height, width
    grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]), [1, grid_shape[1], 1, 1])
    grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]), [grid_shape[0], 1, 1, 1])
    grid = K.concatenate([grid_x, grid_y])
    grid = K.cast(grid, 'float32') # K.cast(grid, K.dtype(feats))

    feats = K.reshape(
        feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    # Adjust preditions to each spatial grid point and anchor size.
    # edited by instructions in https://stackoverflow.com/questions/57558476/training-a-keras-model-yields-multiple-optimizer-errors
    box_xy = (K.sigmoid(feats[..., :2]) + grid) / K.cast(grid_shape[...,::-1], K.dtype(feats))
    box_wh = K.exp(feats[..., 2:4]) * anchors_tensor / K.cast(input_shape[...,::-1], K.dtype(feats))
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_class_probs = K.sigmoid(feats[..., 5:])

    return box_xy, box_wh, box_confidence, box_class_probs

def yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape):
    '''Get corrected boxes'''
    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]
    input_shape = K.cast(input_shape, K.dtype(box_yx))
    image_shape = K.cast(image_shape, K.dtype(box_yx))
    new_shape = K.round(image_shape * K.min(input_shape/image_shape))
    offset = (input_shape-new_shape)/2./input_shape
    scale = input_shape/new_shape
    box_yx = (box_yx - offset) * scale
    box_hw *= scale

    box_mins = box_yx - (box_hw / 2.)
    box_maxes = box_yx + (box_hw / 2.)
    boxes =  K.concatenate([
        box_mins[..., 0:1],  # y_min
        box_mins[..., 1:2],  # x_min
        box_maxes[..., 0:1],  # y_max
        box_maxes[..., 1:2]  # x_max
    ])

    # Scale boxes back to original image shape.
    boxes *= K.concatenate([image_shape, image_shape])
    return boxes

def yolo_boxes_and_scores(feats, anchors, num_classes, input_shape, image_shape):
    '''Process Conv layer output'''
    box_xy, box_wh, box_confidence, box_class_probs = yolo_head_nonLoss(feats,
        anchors, num_classes, input_shape)
    boxes = yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape)
    boxes = K.reshape(boxes, [-1, 4])
    box_scores = box_confidence * box_class_probs
    box_scores = K.reshape(box_scores, [-1, num_classes])
    return boxes, box_scores

def yolo_eval(yolo_outputs,
              anchors,
              num_classes,
              image_shape,
              max_boxes=20,
              score_threshold=.6,
              iou_threshold=.5):
    """Evaluate YOLO model on given input and return filtered boxes."""
    num_layers = len(yolo_outputs)
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [0,1,2]] # default setting
    input_shape = K.shape(yolo_outputs[0])[1:3] * 32
    boxes = []
    box_scores = []
    for l in range(num_layers):
        _boxes, _box_scores = yolo_boxes_and_scores(yolo_outputs[l],
            anchors[anchor_mask[l]], num_classes, input_shape, image_shape)
        boxes.append(_boxes)
        box_scores.append(_box_scores)
    boxes = K.concatenate(boxes, axis=0)
    box_scores = K.concatenate(box_scores, axis=0)
#     print("Box tensor:", boxes.shape)
#     print(K.get_value(boxes))
#     print("Box score tensor:", box_scores.shape)
#     print(K.get_value(box_scores))
#     print("class index of maximum prob value:", K.argmax(box_scores, axis=1).shape)
#     print(K.get_value(K.argmax(box_scores, axis=1)))
#     print("maximum prob value: ",K.max(box_scores, axis=1).shape)
#     print(K.get_value(K.max(box_scores, axis=1)))
#     print("maximum prob mask: ", (K.max(box_scores, axis=1) >= score_threshold).shape)
#     print(K.get_value(K.max(box_scores, axis=1) >= score_threshold)) # 0.027))
#     print()
#     print("index of highest maximum prob value:",K.get_value(K.argmax(K.max(box_scores, axis=1))))
#     print("highest maximum prob value:", K.get_value(K.max(box_scores)))
#     print("class index:", K.get_value(K.argmax(box_scores, axis=1))[K.get_value(K.argmax(K.max(box_scores, axis=1)))])
#     print("raw coordinates:", K.get_value(boxes)[K.get_value(K.argmax(K.max(box_scores, axis=1)))])
#     print("class index:", K.get_value(box_scores)[K.get_value(K.argmax(K.max(box_scores, axis=1)))])
    boxes_dummy = []
    scores_dummy = []
    classes_dummy = []
    scoremask_dummy = K.get_value(K.max(box_scores, axis=1) >= score_threshold)
    for i in range(scoremask_dummy.size):
        if scoremask_dummy[i]:
            boxes_dummy.append(K.get_value(boxes)[i].tolist())
            scores_dummy.append(K.get_value(K.max(box_scores, axis=1))[i].tolist())
            classes_dummy.append(K.get_value(K.argmax(box_scores, axis=1))[i].tolist())
    ''' # toggle to utilize either hacky solution with working classes output or IOU result-cutting solution with non-working classes output
    boxes_ = []
    scores_ = []
    classes_ = []
    mask = box_scores >= score_threshold
    max_boxes_tensor = K.constant(max_boxes, dtype='int32')
    for c in range(num_classes):
        # TODO: use keras backend instead of tf.
        class_boxes = tf.boolean_mask(boxes, mask[:, c])
        class_box_scores = tf.boolean_mask(box_scores[:, c], mask[:, c])
        nms_index = tf.image.non_max_suppression(
            class_boxes, class_box_scores, max_boxes_tensor, iou_threshold=iou_threshold)
        class_boxes = K.gather(class_boxes, nms_index)
        class_box_scores = K.gather(class_box_scores, nms_index)
        classes = K.ones_like(class_box_scores, 'int32') * c
        boxes_.append(class_boxes)
        scores_.append(class_box_scores)
        classes_.append(classes)
    boxes_ = K.concatenate(boxes_, axis=0)
    scores_ = K.concatenate(scores_, axis=0)
    classes_ = K.concatenate(classes_, axis=0)
    '''
    boxes_ = boxes_dummy
    scores_ = scores_dummy
    classes_ = classes_dummy
    '''
    # '''

    return boxes_, scores_, classes_

In [5]:
def inferenceProcessor(jsonModel, fileOutput, imgFolderSource, countNotification=1000, boundBox=False):
    loadedModel = None
    try:
        with open(jsonModel, "r") as jsonFile:
            jsonRead = jsonFile.read()
            # NOTE: this *will* take time, may cause OOM warnings
            loadedModel = model_from_json(jsonRead, custom_objects=customObjects)
            copyLoadModel = clone_model(loadedModel)
            loadedModel = copyLoadModel
            del copyLoadModel
        print("Model loading successful")
        loadedModel.load_weights("./saved_models/model_0_trainModel.h5", by_name=False)    
        print("Model weight loading attempt successful")
    except Exception as e:
        print("Failed to load model/weight data:", e)
    if loadedModel is not None:
        count = 0
        outputFile = open(fileOutput, "a")
        for folderPath in imgFolderSource:
            if count is not 0:
                print("Switching to another folder")
            with os.scandir(folderPath) as folder:
                for entry in folder:
                    if entry.name.endswith(".jpg") and entry.is_file():
                        fileName = str(entry.path)
                        prepped_image = prepare_image(entry.path, (448,448))
                        processedOutput = yolo_eval(
                            yolo_outputs = loadedModel.predict(
                                x = np.expand_dims(prepped_image[0], axis=0),
                                verbose=0,
                                steps=1
                            ),
                            anchors=tinyYolo_anchors,
                            num_classes=80,
                            image_shape=prepped_image[1], # image shape IS NOT model input shape, which is (448, 448)
                            max_boxes=20,
                            score_threshold=0.7,
                            iou_threshold=0.5
                        )
                        ''' 
                            processedOutput = [
                                [each inference's bounding box with format [top, left, bottom, right] in a list],
                                [each inference's score in a list],
                                [each inference's class in a list],
                            ] 
                            # len(processedOutput) is always 3
                            # len(processedOutput[0]) == len(processedOutput[1]) == len(processedOutput[2]) == number of inference
                            # len(processedOutput[0][k]) is always 4 for valid k
                        '''
                        if boundBox:
                            for eachBBox in processedOutput[0]:
                                eachBBox[0] = max(0.0, eachBBox[0]) # bound top to 0
                                eachBBox[1] = max(0.0, eachBBox[1]) # bound left to 0
                                eachBBox[2] = min(prepped_image[1][0] / 1.0, eachBBox[2]) # bound bottom to image height
                                eachBBox[3] = min(prepped_image[1][1] / 1.0, eachBBox[3]) # bound right to image width
                        outputFile.write(json.dumps([fileName, processedOutput]) + "\n")
                        count = count + 1
                        if count % countNotification == 0:
                            print("# of image processed:", count)        
        outputFile.close()
        print(count, "images successfully processed")

In [6]:
for modelPath in ["./saved_models/model_0_inferenceModel.json", "./saved_models/model_1_inferenceModel.json", "./saved_models/model_5_inferenceModel.json"]:
    print("Using inference model:", modelPath)
    inferenceProcessor(
        jsonModel=modelPath,
        fileOutput="./"+modelPath[len("./saved_models/"):len("./saved_models/model_0")]+"_inferResult_bound.txt",
        imgFolderSource=["./train2017/", "./test2017/"],
        boundBox=True
    )

Using inference model: ./saved_models/model_0_inferenceModel.json
Model loading successful
Model weight loading attempt successful
# of image processed: 1000
# of image processed: 2000
# of image processed: 3000
# of image processed: 4000
# of image processed: 5000
# of image processed: 6000
# of image processed: 7000
# of image processed: 8000
# of image processed: 9000
# of image processed: 10000
# of image processed: 11000
# of image processed: 12000
# of image processed: 13000
# of image processed: 14000
# of image processed: 15000
# of image processed: 16000
# of image processed: 17000
# of image processed: 18000
# of image processed: 19000
# of image processed: 20000
# of image processed: 21000
# of image processed: 22000
# of image processed: 23000
# of image processed: 24000
# of image processed: 25000
# of image processed: 26000
# of image processed: 27000
# of image processed: 28000
# of image processed: 29000
# of image processed: 30000
# of image processed: 31000
# of image

# of image processed: 121000
# of image processed: 122000
# of image processed: 123000
# of image processed: 124000
# of image processed: 125000
# of image processed: 126000
# of image processed: 127000
# of image processed: 128000
# of image processed: 129000
# of image processed: 130000
# of image processed: 131000
# of image processed: 132000
# of image processed: 133000
# of image processed: 134000
# of image processed: 135000
# of image processed: 136000
# of image processed: 137000
# of image processed: 138000
# of image processed: 139000
# of image processed: 140000
# of image processed: 141000
# of image processed: 142000
# of image processed: 143000
# of image processed: 144000
# of image processed: 145000
# of image processed: 146000
# of image processed: 147000
# of image processed: 148000
# of image processed: 149000
# of image processed: 150000
# of image processed: 151000
# of image processed: 152000
# of image processed: 153000
# of image processed: 154000
# of image pro